# Function arguments in Python

In this lecture, we will explore the four types of functional arguments in Python, namely

1. Position-only arguments (only in Python 3.8--not in Anaconda as of Feb. 2021).
    1. *Skipping for now*
2. Positional or keyword arguments.
3. Varargs (a variable number of extra arguments).
4. Keyword only arguments.
5. kwargs (a variable number of extra keyword arguments)

## Note on position-only arguments.

A new type of argument was introduced in Python 3.8, but Anaconda hasn't been upgraded yet due to some incompatibility with certain packages.  We will be ignoring this type of argument for now.  Expect this lecture to be updated in the future.

In [1]:
import sys

In [2]:
sys.version

'3.9.12 (main, Jun  1 2022, 06:34:44) \n[Clang 12.0.0 ]'

## Position or keyword arguments.

The most common type of arguments can be accessed either through their position or as keywords.  Note that Python requires that arguments with a default value follow those that do not.

In [3]:
def f(pos_or_kw1, pos_or_kw2, pos_or_kw3 = None):
    return f"a1 = {pos_or_kw1}, a2 = {pos_or_kw2}, a3 = {pos_or_kw3}"

#### Non-default via position

In [4]:
f(1, 2)

'a1 = 1, a2 = 2, a3 = None'

#### Non-default via keyword

In [5]:
f(pos_or_kw1=1, pos_or_kw2=2)

'a1 = 1, a2 = 2, a3 = None'

#### keyword arguments can be in any order

In [6]:
f(pos_or_kw2=1, pos_or_kw1=2)

'a1 = 2, a2 = 1, a3 = None'

#### Args without defaults are required

In [7]:
f(1)

TypeError: f() missing 1 required positional argument: 'pos_or_kw2'

#### Default via position

In [8]:
f(1, 2, 3)

'a1 = 1, a2 = 2, a3 = 3'

#### Default via keyword

In [9]:
f(pos_or_kw1=1, pos_or_kw2=2, pos_or_kw3=3)

'a1 = 1, a2 = 2, a3 = 3'

#### Again keyword arguments can be in any order

In [10]:
f(pos_or_kw3=1, pos_or_kw2=2, pos_or_kw1=3)

'a1 = 3, a2 = 2, a3 = 1'

## Varargs

We can add a variable number of additional arguments using the `*args` arguments.  The arguments for these additiona entries will be stored in a tuple named `args`.  

Note that varargs must follow all position or keyword arguments.

In [11]:
def g(p_kw1, p_kw2, p_kw3 = None, *args):
    return f"a1 = {p_kw1}, a2 = {p_kw2}, a3 = {p_kw3}, star_args = {args}"

#### Positions for defaults are preserved.

Note that the third position provides a value to `p_kw3` then the remaining arguments get passed to `args`

In [12]:
g(1, 2, 3, 4, 5)

'a1 = 1, a2 = 2, a3 = 3, star_args = (4, 5)'

#### `args` is just another name

While it is convential to use `args` in `*args`, the choice of name is up to the programmer.

In [13]:
def g2(*my_args):
    return my_args

In [14]:
g2(1, 2, "a", "b")

(1, 2, 'a', 'b')

## Keyword-only parameters

Any parameters defined after `*args` will be deemed *keyword-only* and can only be accessed through keyword-assignment.

In [15]:
def h(p_kw1, p_kw2, p_kw3 = None, *args, kw_only = "> Silas"):
    return f"a1 = {p_kw1}, a2 = {p_kw2}, a3 = {p_kw3}, star_args = {args}, Iverson {kw_only}"

#### Using the default value

In [17]:
h(1, 2, 3, 4, 5)

'a1 = 1, a2 = 2, a3 = 3, star_args = (4, 5), Iverson > Silas'

#### Changing the default with keyword-assignment

In [18]:
h(1, 2, 3, 4, 5, kw_only = "!= Malone")

'a1 = 1, a2 = 2, a3 = 3, star_args = (4, 5), Iverson != Malone'

#### Defining keyword-only parameters without varargs.

If you would like to define keyword-only parameters **but no varargs**, insert a `*,` after the last `positional or keyword` parameter.

In [19]:
def h2(p_kw1, p_kw2, p_kw3 = None, *, kw_only = "> Silas"):
    return f"a1 = {p_kw1}, a2 = {p_kw2}, a3 = {p_kw3}, Iverson {kw_only}"

#### Using both the default value

In [20]:
h2(1, 2)

'a1 = 1, a2 = 2, a3 = None, Iverson > Silas'

#### Keeping the default value for `kw_only`

In [21]:
h2(1, 2, 3)

'a1 = 1, a2 = 2, a3 = 3, Iverson > Silas'

#### Can't access `kw_only` via a positional argument.

In [22]:
h2(1, 2, 3, 4)

TypeError: h2() takes from 2 to 3 positional arguments but 4 were given

#### Must use keyword assignment to change `kw_only`

In [23]:
h2(1, 2, 3, kw_only="< Hooks")

'a1 = 1, a2 = 2, a3 = 3, Iverson < Hooks'

## A variable number of additional keyword arguments

Finally, we can use `**kwargs` to gather any number of additional keyword-only arguments.  The resulting values will be stored in a `dict` with keywords as keys and argument values as values.  Again, the `kwargs` name is customary, but can be changed by the programmer.

In [24]:
def m(p1, p2, *, kw1 = None, **my_kwargs):
    return my_kwargs

In [25]:
m(1, 2, Iverson="Great", Bergen="likes R")

{'Iverson': 'Great', 'Bergen': 'likes R'}

# Impact on `pipeable` functions

When creating `pipeable` functions, we need to make a couple adjustments.

1. We **CANNOT** mix `*args` and curried functions, as the function will never complete.
    * **Solution.** Add `cols` as a kwarg.
2. Consider switching all parameters with default values to keyword-only parameters.
3. Use `**kwargs` as a way to allow for variable number of inputs.

In [26]:
!pip install composable

In [38]:
import pandas as pd
from composable import pipeable

## Example -- Making a pipeable string `slice`

For example, suppose we want to make a pipeable version of `panda`s `str.slice` method.  Before implementing the method, we should make sure we understand the method by constructing a number of examples.  For illustrations sake, I have provided one below.

In [44]:
from composable.strict import map
c = pd.Series(map(str, range(1000,1011)))
c

0     1000
1     1001
2     1002
3     1003
4     1004
5     1005
6     1006
7     1007
8     1008
9     1009
10    1010
dtype: object

In [50]:
c.str.slice(1) #start slice at index 1

0     000
1     001
2     002
3     003
4     004
5     005
6     006
7     007
8     008
9     009
10    010
dtype: object

In [51]:
c.str.slice(1, 3) # slice from index 1 to 3

0     00
1     00
2     00
3     00
4     00
5     00
6     00
7     00
8     00
9     00
10    01
dtype: object

In [56]:
c.str.slice(0, 4, 2) # slice from 0 to 4 in steps of 2

0     10
1     10
2     10
3     10
4     10
5     10
6     10
7     10
8     10
9     10
10    11
dtype: object

### Inspecting the signature with `help`

Next, we need to inspect the formal parameters and watch out for problems (variable args, position only, position-or-keyword).

Inspecting the help for this method shows the following signature.

```
slice(self, start=None, stop=None, step=None)
```

In [ ]:
help(pd.Series.str.slice)

Since we are switching from a method to a function, `self` will be removed and replaced with an argument for the incoming column, which we will call `col`

### Naive approach

First, we illustrate the need for careful adjustment of the parameters by naively implementing the original arguments, with the only change being renaming `self` to `col`

In [48]:
@pipeable
def slice(col, start=None, stop=None, step=None):
    return col.str.slice( start = start, stop = stop, step = step)

In [58]:
c >> slice(1, 3) # first argument passed to `col` by position.

AttributeError: 'int' object has no attribute 'str'

### What went wrong?

Since the optional arguments are position-or-keyword, they can mess-up the piping as `col` get filled by the first argument and then the actual column gets piped into a later position. 

### Option 1 -- Make all optional parameters keyword-only

One solution is to switch all position-or-keyword arguments to keyword only arguments.  While this leads to longer code--we are forced to write out each argument as a keyword--it could be argued that this also leads to more readable code.

In [64]:
@pipeable
def slice(col, *, start=None, stop=None, step=None):
    return col.str.slice(start, stop, step)

In [65]:
c >> slice(start = 1, stop = 3) # second and third digits

0     00
1     00
2     00
3     00
4     00
5     00
6     00
7     00
8     00
9     00
10    01
dtype: object

### Option 2 -- Use position-only arguments

The other option is to 

1. Make all arguments position only
2. Move `self/col` to the last position.

This has the advantage of removing the required keyword arguments, but the trade-off is a requirement to *alway* specifiy all three arguments.

In [ ]:
@pipeable
def slice(start, stop, step, col, /):
    return col.str.slice(start, stop, step)

In [62]:
c >> slice(1,3,1)

0     00
1     00
2     00
3     00
4     00
5     00
6     00
7     00
8     00
9     00
10    01
dtype: object

## Main takeaways

The main points are that

* position-or-keyword arguments and regular keyword arguments cause problems with piping as they can change the location we pipe into.
* var-args never work with piping as we cannot determine when to stop.
* position-only, keyword-only, and variable keyword arguments all work well with pipes.